In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-airline-sentiment/Tweets.csv
/kaggle/input/twitter-airline-sentiment/database.sqlite


In [ ]:
df1 = pd.read_csv("/kaggle/input/twitter-airline-sentiment/Tweets.csv")
df1

df2 = df1

In [ ]:
df1.describe()

,tweet_id,airline_sentiment_confidence,negativereason_confidence,retweet_count
count,1.464000e+04,14640.000000,10522.000000,14640.000000
mean,5.692184e+17,0.900169,0.638298,0.082650
std,7.791112e+14,0.162830,0.330440,0.745778
min,5.675883e+17,0.335000,0.000000,0.000000
25%,5.685592e+17,0.692300,0.360600,0.000000
50%,5.694779e+17,1.000000,0.670600,0.000000
75%,5.698905e+17,1.000000,1.000000,0.000000
max,5.703106e+17,1.000000,1.000000,44.000000


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [ ]:
df1 = df1[['airline_sentiment', 'text']]

In [ ]:
df = df1
df['text'] = df['text'].str.lower()
df['airline_sentiment']  = df['airline_sentiment'].str.lower()
df

/tmp/ipykernel_18/1729734181.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].str.lower()
/tmp/ipykernel_18/1729734181.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['airline_sentiment']  = df['airline_sentiment'].str.lower()


,airline_sentiment,text
0,neutral,@virginamerica what @dhepburn said.
1,positive,@virginamerica plus you've added commercials t...
2,neutral,@virginamerica i didn't today... must mean i n...
3,negative,@virginamerica it's really aggressive to blast...
4,negative,@virginamerica and it's a really big bad thing...
...,...,...
14635,positive,@americanair thank you we got on a different f...
14636,negative,@americanair leaving over 20 minutes late flig...
14637,neutral,@americanair please bring american airlines to...
14638,negative,"@americanair you have my money, you change my ..."


In [ ]:
# Example contractions dictionary
contractions_dict = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "couldn't": "could not",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'll": "he will",
  "here's": "here is",
  "how's": "how is",
  "i'd": "i would",
  "i'll": "i will",
  "i'm": "i am",
  "isn't": "is not",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mightn't": "might not",
  "mustn't": "must not",
  "shan't": "shall not",
  "she'd": "she would",
  "she'll": "she will",
  "that's": "that is",
  "there's": "there is",
  "they'd": "they would",
  "they're": "they are",
  "they've": "they have",
  "this's": "this is",
  "wasn't": "was not",
  "we'd": "we would",
  "we'll": "we will",
  "weren't": "were not",
  "what's": "what is",
  "when's": "when is",
  "where's": "where is",
  "who's": "who is",
  "why's": "why is",
  "won't": "will not",
  "you'd": "you would",
  "you'll": "you will",
  "you're": "you are",
    "you've": "you have"
}

def expand_contractions(text, contractions_dict):
  """
  Expands contractions in text using a dictionary.

  Args:
    text: The text to be processed.
    contractions_dict: A dictionary of contractions and their expansions.

  Returns:
    The expanded text.
  """
  expanded_text = []
  for word in text.split():
    if word.lower() in contractions_dict:
      expanded_text.append(contractions_dict[word.lower()])
    else:
      expanded_text.append(word)
  return " ".join(expanded_text)

In [ ]:
df['text'] = df['text'].apply(expand_contractions, args=(contractions_dict,))
df

/tmp/ipykernel_18/554856839.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(expand_contractions, args=(contractions_dict,))


,airline_sentiment,text
0,neutral,@virginamerica what @dhepburn said.
1,positive,@virginamerica plus you have added commercials...
2,neutral,@virginamerica i did not today... must mean i ...
3,negative,@virginamerica it is really aggressive to blas...
4,negative,@virginamerica and it is a really big bad thin...
...,...,...
14635,positive,@americanair thank you we got on a different f...
14636,negative,@americanair leaving over 20 minutes late flig...
14637,neutral,@americanair please bring american airlines to...
14638,negative,"@americanair you have my money, you change my ..."


In [ ]:
from nltk.tokenize import word_tokenize
# df['text'] = df['text'].str.lower()
df['text_tokened'] = df['text'].apply(word_tokenize)
df

/tmp/ipykernel_18/7797780.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_tokened'] = df['text'].apply(word_tokenize)


,airline_sentiment,text,text_tokened
0,neutral,@virginamerica what @dhepburn said.,"[@, virginamerica, what, @, dhepburn, said, .]"
1,positive,@virginamerica plus you have added commercials...,"[@, virginamerica, plus, you, have, added, com..."
2,neutral,@virginamerica i did not today... must mean i ...,"[@, virginamerica, i, did, not, today, ..., mu..."
3,negative,@virginamerica it is really aggressive to blas...,"[@, virginamerica, it, is, really, aggressive,..."
4,negative,@virginamerica and it is a really big bad thin...,"[@, virginamerica, and, it, is, a, really, big..."
...,...,...,...
14635,positive,@americanair thank you we got on a different f...,"[@, americanair, thank, you, we, got, on, a, d..."
14636,negative,@americanair leaving over 20 minutes late flig...,"[@, americanair, leaving, over, 20, minutes, l..."
14637,neutral,@americanair please bring american airlines to...,"[@, americanair, please, bring, american, airl..."
14638,negative,"@americanair you have my money, you change my ...","[@, americanair, you, have, my, money, ,, you,..."


In [ ]:
# To replace the mention with people or airline, we need the list of airlines, we can grab airlines from the first daaframe
airlines = df2['airline'].str.lower().unique().tolist()
airlines

['virgin america', 'united', 'southwest', 'delta', 'us airways', 'american']

In [ ]:
def replace_with_person_or_airline(text_tokens, airlines_dict):
    for i in range(len(text_tokens)):
        if text_tokens[i] == '@' and text_tokens[i+1] in airlines_dict:
            text_tokens[i+1] = "airline"
            i += 2  # Skip next word as well
        elif text_tokens[i] == '@':
            text_tokens[i+1] = "person"
            i += 2  # Skip next word as well
    return text_tokens

In [ ]:
df['cleaned'] = df['text_tokened'].apply(replace_with_person_or_airline,args=(airlines,))
df['cleaned']

/tmp/ipykernel_18/1726309089.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned'] = df['text_tokened'].apply(replace_with_person_or_airline,args=(airlines,))


0                    [@, person, what, @, person, said, .]
1        [@, person, plus, you, have, added, commercial...
2        [@, person, i, did, not, today, ..., must, mea...
3        [@, person, it, is, really, aggressive, to, bl...
4        [@, person, and, it, is, a, really, big, bad, ...
                               ...                        
14635    [@, person, thank, you, we, got, on, a, differ...
14636    [@, person, leaving, over, 20, minutes, late, ...
14637    [@, person, please, bring, american, airlines,...
14638    [@, person, you, have, my, money, ,, you, chan...
14639    [@, person, we, have, 8, ppl, so, we, need, 2,...
Name: cleaned, Length: 14640, dtype: object

In [ ]:
df['cleaned'] = df['text_tokened'].apply(lambda x:[y for y in x if y != '@'])
df['cleaned']

/tmp/ipykernel_18/2808321106.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned'] = df['text_tokened'].apply(lambda x:[y for y in x if y != '@'])


0                          [person, what, person, said, .]
1        [person, plus, you, have, added, commercials, ...
2        [person, i, did, not, today, ..., must, mean, ...
3        [person, it, is, really, aggressive, to, blast...
4        [person, and, it, is, a, really, big, bad, thi...
                               ...                        
14635    [person, thank, you, we, got, on, a, different...
14636    [person, leaving, over, 20, minutes, late, fli...
14637    [person, please, bring, american, airlines, to...
14638    [person, you, have, my, money, ,, you, change,...
14639    [person, we, have, 8, ppl, so, we, need, 2, kn...
Name: cleaned, Length: 14640, dtype: object

In [ ]:
from keras.preprocessing.text import Tokenizer

# Create a tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['cleaned'])

# Convert text to sequences of numbers
sequences = tokenizer.texts_to_sequences(df['cleaned'])
vocab_size = len(tokenizer.index_word)
# number of unique words in dataset
vocab_size

2024-02-28 18:48:40.623196: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 18:48:40.623356: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 18:48:40.796985: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


16464

In [ ]:
from keras.utils import pad_sequences
sequences = pad_sequences(sequences, padding='pre')
sequences.shape

(14640, 46)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df['airline_sentiment_labelencoded'] = le.fit_transform(df['airline_sentiment'])

/tmp/ipykernel_18/1138143624.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['airline_sentiment_labelencoded'] = le.fit_transform(df['airline_sentiment'])


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sequences, df['airline_sentiment_labelencoded'], test_size=0.2, random_state=42)

## Simple RNN

input_dim = size of the vocabulary
input_length = 46 words (if the tweet was shorter, it was padded. If longer, it is truncated
output_dim = 2

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Dense, Bidirectional, SimpleRNN,GRU,LSTM,Dropout

model = Sequential([
    Embedding(input_dim=16465,output_dim=2,input_length=46),
    Bidirectional(SimpleRNN(53)),
    Dropout(0.5),
    Dense(12,activation='relu'),
    Dense(3,activation='softmax')
])

#compiling model
model.compile(optimizer="rmsprop",loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 46, 2)             32930     
                                                                 
 bidirectional (Bidirection  (None, 106)               5936      
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 106)               0         
                                                                 
 dense (Dense)               (None, 12)                1284      
                                                                 
 dense_1 (Dense)             (None, 3)                 39        
                                                                 
Total params: 40189 (156.99 KB)
Trainable params: 40189 (156.99 KB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [ ]:
history = model.fit(X_train, y_train, epochs=16, validation_data = (X_test[:-200], y_test[:-200]))

Epoch 1/16
366/366 [==============================] - 7s 15ms/step - loss: 0.8683 - accuracy: 0.6160 - val_loss: 0.7313 - val_accuracy: 0.6844
Epoch 2/16
366/366 [==============================] - 5s 14ms/step - loss: 0.7003 - accuracy: 0.6966 - val_loss: 0.6573 - val_accuracy: 0.7324
Epoch 3/16
366/366 [==============================] - 5s 14ms/step - loss: 0.6258 - accuracy: 0.7299 - val_loss: 0.6094 - val_accuracy: 0.7434
Epoch 4/16
366/366 [==============================] - 5s 14ms/step - loss: 0.5802 - accuracy: 0.7534 - val_loss: 0.6435 - val_accuracy: 0.7485
Epoch 5/16
366/366 [==============================] - 5s 14ms/step - loss: 0.5439 - accuracy: 0.7750 - val_loss: 0.5620 - val_accuracy: 0.7746
Epoch 6/16
366/366 [==============================] - 5s 14ms/step - loss: 0.5137 - accuracy: 0.7906 - val_loss: 0.5644 - val_accuracy: 0.7738
Epoch 7/16
366/366 [==============================] - 5s 14ms/step - loss: 0.4919 - accuracy: 0.8047 - val_loss: 0.5805 - val_accuracy: 0.7672

In [ ]:
# Predict on the test data
predictions = model.predict(X_test[-200:])
# Now evaluate the model using metrics like accuracy, precision, recall, etc.
from sklearn.metrics import accuracy_score, classification_report

# Convert predictions to class labels (assuming predictions are in one-hot encoded format)
predicted_labels = predictions.argmax(axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test[-200:], predicted_labels) # GRADING, COMPARING YOUR ANSWERS TO SOLUTION KEY
print(f"Accuracy: {accuracy}")

7/7 [==============================] - 0s 4ms/step
Accuracy: 0.855
